In [27]:
!pip install torchdata   # only rank 0 needs to do this the first time


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 KB 7.8 kB/s eta 0:00:0000:01


In [9]:
import pandas as pd
csv_path="/rsrch1/ip/msalehjahromi/codes/FineTune/nlst_event_train_val_test_.csv"
 
df=pd.read_csv(csv_path)#.query("split == 'train'")
set(df["split"].to_list())# {'test', 'train', 'val'}
I wanna save a nlst_event_train_val_ in a way that all rows with split of val changes to split of train , and the test one changes to val.

{'test', 'train', 'val'}

In [10]:

# 2. Remap splits: val→train, test→val (train stays train)
df['split'] = df['split'].replace({
    'val':   'train',
    'test':  'val'
})

# 3. (Optional) Check you only have 'train' and 'val' now
print(set(df['split']))  # -> {'train', 'val'}

# 4. Save to a new CSV
out_path = "/rsrch1/ip/msalehjahromi/codes/FineTune/nlst_event_train_val_.csv"
df.to_csv(out_path, index=False)

print(f"Saved remapped splits to {out_path}")


{'val', 'train'}
Saved remapped splits to /rsrch1/ip/msalehjahromi/codes/FineTune/nlst_event_train_val_.csv


In [28]:
# count each split
counts = df['split'].value_counts()
print(counts)


split
train    32922
test      5470
val       5460
Name: count, dtype: int64


In [12]:
df

,pid,gender,age,pkyr,cigsmok,diagcopd,famfather,scr_res0,scr_res1,scr_res2,...,time,time_years,1-year-cancer,2-year-cancer,3-year-cancer,4-year-cancer,5-year-cancer,6-year-cancer,split,file_path
0,111632_T0,1,56,51.25,1,0.0,0.0,3,2,2,...,1224.0,3.351129,0,0,0,-1,-1,-1,train,/rsrch7/home/ip_rsrch/wulab/Lung_Foundation_Mo...
1,110025_T0,1,71,90.00,0,0.0,0.0,2,2,5,...,572.0,1.566051,0,-1,-1,-1,-1,-1,train,/rsrch7/home/ip_rsrch/wulab/Lung_Foundation_Mo...
2,120677_T0,1,58,80.00,1,0.0,1.0,2,3,2,...,935.0,2.559890,0,0,-1,-1,-1,-1,train,/rsrch7/home/ip_rsrch/wulab/Lung_Foundation_Mo...
3,121403_T0,1,56,54.00,1,0.0,0.0,2,1,3,...,2001.0,5.478439,0,0,0,0,0,-1,val,/rsrch7/home/ip_rsrch/wulab/Lung_Foundation_Mo...
4,129141_T0,1,55,80.00,1,0.0,0.0,4,6,5,...,1984.0,5.431896,0,0,0,0,0,-1,train,/rsrch7/home/ip_rsrch/wulab/Lung_Foundation_Mo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43847,200508_T2,1,67,114.00,1,0.0,NaN,4,6,6,...,1784.0,4.882957,0,0,0,0,-1,-1,train,/rsrch7/home/ip_rsrch/wulab/Lung_Foundation_Mo...
43848,204854_T2,2,61,54.00,0,0.0,0.0,4,4,5,...,1989.0,5.444216,0,0,0,0,0,-1,train,/rsrch7/home/ip_rsrch/wulab/Lung_Foundation_Mo...
43849,205247_T2,2,61,32.25,1,0.0,0.0,4,4,5,...,1995.0,5.460643,0,0,0,0,0,-1,train,/rsrch7/home/ip_rsrch/wulab/Lung_Foundation_Mo...
43850,211365_T2,2,59,40.00,1,0.0,0.0,1,3,2,...,1981.0,5.422313,0,0,0,0,0,-1,val,/rsrch7/home/ip_rsrch/wulab/Lung_Foundation_Mo...


In [16]:
from typing import Dict, Any, List, Optional, Callable
class VolumeProcessor:
    def __init__(self, chunk_depth=3, out_size=(448, 448), vmin=-1000, vmax=150, eps=0.00005):
        self.chunk_depth = chunk_depth
        self.out_size = out_size
        self.vmin = vmin
        self.vmax = vmax
        self.eps = eps
    
    def _preprocess_volume(self, nifti_path):
        # Load NIfTI file
        nifti = nib.load(nifti_path)
        volume = nifti.get_fdata()
        
        # Normalize to 0-1 range
        volume = (volume - volume.min()) / (volume.max() - volume.min() + 1e-8)
        
        return volume
    
    def _get_chunks(self, volume):
        # Extract 3-slice chunks with stride 1
        depth, height, width = volume.shape
        
        # Create list of chunks
        chunks = []
        for i in range(0, depth - self.chunk_depth + 1):
            chunk = volume[i:i+self.chunk_depth]  # Get 3 consecutive slices
            
            # Resize if needed
            if (height, width) != self.out_size:
                chunk_resized = np.zeros((self.chunk_depth, *self.out_size))
                for j in range(self.chunk_depth):
                    # Use CV2 or other resize method here
                    # For simplicity, this is a placeholder
                    chunk_resized[j] = chunk[j]  # Replace with actual resize
                chunk = chunk_resized
            
            # Create "RGB" channels by duplicating (3D to pseudo-RGB)
            chunk_rgb = np.stack([chunk] * 3, axis=1)  # [3, 3, H, W]
            chunks.append(chunk_rgb)
        
        return chunks
    
    def process_file(self, nifti_path):
        volume = self._preprocess_volume(nifti_path)
        chunks = self._get_chunks(volume)
        return chunks

class NLSTDataset(Dataset):
    def __init__(self,
                 df: pd.DataFrame,
                 processor: VolumeProcessor,
                 label_cols: List[str],
                 augment: Optional[Callable[[torch.Tensor], torch.Tensor]] = None):
        df = df[df.file_path.notna()].reset_index(drop=True)
        self.df = df
        self.proc = processor
        self.labels = label_cols
        self.augment = augment

    def __len__(self) -> int:
        return len(self.df)

    def __getitem__(self, idx: int, from_rsrch1: bool = True):
        row = self.df.iloc[idx]  # Shape: pandas.Series (1D row data)
        nii_path = row.file_path  # Shape: str (file path)
        if from_rsrch1:
            old = "/rsrch7/home/ip_rsrch/wulab/Lung_Foundation_Model_Data_/Batchs_Nii/Vols/NLST"
            new = "/rsrch1/ip/msalehjahromi/data/NLST"
            nii_path = nii_path.replace(old, new)
        #print(f"[Data] Loading volume {idx}: {nii_path}")
        nii = nib.load(nii_path)
        vol = nii.get_fdata().astype(np.float32)  # Shape: [H, W, D] 3D volume
        spacing = nii.header.get_zooms()  # Get voxel spacing (dx, dy, dz)
        H, W, D = vol.shape
        #print(f"[Data] Volume shape: {vol.shape}")
        num_chunks = D // self.proc.chunk_depth  # Shape: scalar (number of chunks)

        windows = []  # Will collect chunks
        for i in range(num_chunks):
            arr = vol[:, :, i*self.proc.chunk_depth:(i+1)*self.proc.chunk_depth]  # Shape: [H, W, chunk_depth]
            arr = np.clip(arr, self.proc.vmin, self.proc.vmax)  # Shape: [H, W, chunk_depth]
            arr = (arr - self.proc.vmin) / (self.proc.vmax - self.proc.vmin)  # Shape: [H, W, chunk_depth]
            arr = np.clip(arr, self.proc.eps, 1.0 - self.proc.eps)  # Shape: [H, W, chunk_depth]

            t = torch.from_numpy(arr).permute(2, 0, 1)  # Shape: [chunk_depth, H, W]
            t = F.interpolate(t.unsqueeze(0), size=self.proc.out_size,
                              mode="bilinear", align_corners=False).squeeze(0)  # Shape: [chunk_depth, 448, 448]
            t = (t - 0.5) / 0.5  # Shape: [chunk_depth, 448, 448]
            windows.append(t)  # Add to list

        if self.augment is not None:
            windows = [self.augment(c) for c in windows]
        chunks = torch.stack(windows, dim=0)  # Shape: [num_chunks, chunk_depth, 448, 448]
        labels = row[self.labels].to_numpy(dtype=np.float32)  # Shape: [num_labels] (e.g., [3] for 3 time points)
        mask = (labels != -1)  # Shape: [num_labels] boolean mask
        return chunks, labels, mask, spacing  # Return shapes: [num_chunks, chunk_depth, 448, 448], [num_labels], [num_labels], (dx, dy, dz)


In [26]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import nibabel as nib
import numpy as np
import torch.nn.functional as F

# --- 1. Load & filter ---
csv_path = "/rsrch1/ip/msalehjahromi/codes/FineTune/nlst_event_train_val_test_.csv"
df = pd.read_csv(csv_path)

label_col = "1-year-cancer"
print(df.shape)
# drop any rows where this label is –1
df = df[df[label_col] != -1].reset_index(drop=True)
print(df.shape)
# --- 2. Compute class counts & weights ---
# value_counts gives e.g. {0: 10000, 1: 500}
counts = df[label_col].value_counts().to_dict()
# invert freq to get per-class weight:
class_weights = {cls: 1.0/count for cls, count in counts.items()}

# assign each sample its class weight
sample_weights = df[label_col].map(class_weights).tolist()
print(len(sample_weights))
# create a sampler that draws N=df.shape[0] samples with replacement
sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),
    replacement=True
)

# --- 3. Plug into your DataLoader ---
# assume you’ve defined NLSTDataset exactly as above, but you pass in the filtered df
label_cols = ['1-year-cancer', '2-year-cancer', '3-year-cancer', '4-year-cancer', '5-year-cancer', '6-year-cancer']
    
processor = VolumeProcessor(chunk_depth=3, out_size=(448, 448))
dataset = NLSTDataset(df, processor, label_cols)
loader = DataLoader(
    dataset,
    batch_size=8,
    sampler=sampler,      # ← balanced sampling instead of shuffle=True
    num_workers=1,
    pin_memory=True
)
print("hello hello")
# now each batch will on average contain roughly equal numbers of 0s and 1s
for chunks, labels, mask, spacing in loader:
    # train your model…
    print(labels, mask, spacing)
    print()
    break



(43852, 21)
(43852, 21)
43852
hello hello


RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/.local/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/.local/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 54, in fetch
    return self.collate_fn(data)
  File "/.local/lib/python3.10/site-packages/torch/utils/data/_utils/collate.py", line 264, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
  File "/.local/lib/python3.10/site-packages/torch/utils/data/_utils/collate.py", line 142, in collate
    return [collate(samples, collate_fn_map=collate_fn_map) for samples in transposed]  # Backwards compatibility.
  File "/.local/lib/python3.10/site-packages/torch/utils/data/_utils/collate.py", line 142, in <listcomp>
    return [collate(samples, collate_fn_map=collate_fn_map) for samples in transposed]  # Backwards compatibility.
  Fil

In [35]:
# def calculate_class_weights(df, label_cols):
#     """Calculate class weights for imbalanced classification"""
#     weights = []
    
#     for col in label_cols:
#         # Count positive and negative samples
#         pos_count = df[col].sum()
#         neg_count = len(df) - pos_count
        
#         # Calculate weight (higher weight for minority class)
#         if pos_count > 0 and neg_count > 0:
#             weight = neg_count / pos_count  # Weight for positive class
#         else:
#             weight = 1.0
        
#         weights.append(weight)
    
#     return torch.tensor(weights)

# import torch

def calculate_class_weights(df, label_cols):
    """Calculate class weights for imbalanced binary labels (only 0 vs 1, ignore -1)."""
    weights = []
    for col in label_cols:
        # Only keep 0/1 entries
        vals = df[col].isin([0, 1])
        pos_count = (df.loc[vals, col] == 1).sum()
        neg_count = (df.loc[vals, col] == 0).sum()

        # If both classes exist, weight positive by neg/pos; else fallback to 1.0
        if pos_count > 0 and neg_count > 0:
            w = neg_count / pos_count
        else:
            w = 1.0

        weights.append(w)

    return torch.tensor(weights, dtype=torch.float32)


csv_path="/rsrch1/ip/msalehjahromi/codes/FineTune/nlst_event_train_val_.csv"
 
df=pd.read_csv(csv_path)#.query("split == 'train'")
label_cols = ['1-year-cancer', '2-year-cancer', '3-year-cancer', '4-year-cancer', '5-year-cancer', '6-year-cancer']
calculate_class_weights(df, label_cols)

tensor([26.1361, 19.0005, 15.1556, 12.7369,  8.5288,  4.3790])

In [32]:
counts = df[label_cols].apply(pd.Series.value_counts).fillna(0).astype(int)

print(counts)

    1-year-cancer  2-year-cancer  3-year-cancer  4-year-cancer  5-year-cancer  \
-1              0            911           1896           3342          12969   
 0          42236          40794          39359          37561          27642   
 1           1616           2147           2597           2949           3241   

    6-year-cancer  
-1          25472  
 0          14963  
 1           3417  


In [34]:
42/1.6, 40/2.1, 39/2.5, 37/2.9, 27/3.2, 14.9/3.4

(26.25,
 19.047619047619047,
 15.6,
 12.758620689655173,
 8.4375,
 4.382352941176471)